In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
from gurobipy import *

In [8]:
df = pd.read_csv("Fife-data/flex_networks.csv")
df["Timestamp"] = pd.to_datetime(df["Timestamp"], infer_datetime_format=True)
load = np.asarray(df["crawfordCrescent_F2"])

In [9]:
time = list(range(48))
soe_time = list(range(48))
C_rate = 8000
soe_init = 0

In [10]:
m = ConcreteModel()
m.cap = Var(domain=Reals, bounds=(10,1000))#battery capacity variable
m.utility = Var(time,domain=NonNegativeReals) #purchase from grid
m.pe_c = Var(time,domain=NonNegativeReals,bounds=(0,1000)) #charge to battery
m.pe_d = Var(time,domain=NonNegativeReals,bounds=(0,1000)) #discharge from battery
m.SOE = Var(soe_time, domain=NonNegativeReals,bounds=(0,1000)) #state of energy inside battery

def obj_expression(m):
    return (m.cap*cost) +  sum([price[i]*(m.utility[i]) for i in time]) 
    m.OBJ = Objective(rule=obj_expression, sense=minimize) 

def battery_rule(m,i):  #to connect the states of the amount of energy stored at, energy discharged from, energy charged to, the battery system
    return m.SOE[i+1] == m.SOE[i] + m.pe_c[i] - m.pe_d[i] 
    m.battery_rule = Constraint(time, rule=battery_rule)

def capacity_rule(m,i):  # a physical constraint that limits the amount of energy stored in the battery system
    return m.SOE[i] <= m.cap
    m.capacity_rule = Constraint(time, rule=capacity_rule)

def charging_rule(m,i):  # a physical constraint that limits the amount of energy charged to the battery system
    return m.pe_c[i] <= m.cap * C_rate
    m.charging_rule = Constraint(time, rule=charging_rule)

def discharging_rule(m,i):  # a physical constraint that limits the amount of energy discharged from the battery system
    return  m.pe_d[i] <= m.cap * C_rate
    m.discharging_rule = Constraint(time, rule=discharging_rule)

def soe_start_rule(m): 
    return m.SOE[0] == soe_init
    m.soe_start_rule = Constraint(rule=soe_start_rule)

def soe_end_rule(m):
    return m.SOE[n] == m.SOE[0]
    m.soe_end = Constraint(rule=soe_end_rule)

def demand_rule(m,i): #a physical constraint, representing the law of conservation of energy
    return m.pe_d[i] - m.pe_c[i]  + m.utility[i]  ==  load[i]
    m.demand_rule = Constraint(time, rule=demand_rule)

def binary_charge_constraint(m,i): #making sure it is not charging while discharging and viceversa
    return m.pe_d[i]*m.pe_c[i]== 0
    m.binary_charge_constraint = Constraint(time, rule=binary_charge_constraint)

instance = m.create_instance()
opt = SolverFactory("gurobi")
opt.options['NonConvex'] = 2
results = opt.solve(instance, tee=True)

TypeError: __init__() takes exactly 3 positional arguments (1 given)